In [10]:
import pandas as pd
import numpy as np
import os
import plotly.express as px

In [13]:
# os.chdir('../')

In [16]:
filename = pd.read_csv('src/raw_data/ground_truth.csv')['file'][0]
recovery = pd.read_csv('src/raw_data/ground_truth.csv')['recovery'][0]
drop = pd.read_csv('src/raw_data/ground_truth.csv')['drop'][0]

df = pd.read_csv(f'src/raw_data/train/{filename}', sep = '\\s+', header = None, names = ['time', 'pressure'])
df

FileNotFoundError: [Errno 2] No such file or directory: 'src/raw_data/ground_truth.csv'

In [17]:
import ast

recovery = ast.literal_eval(recovery)
drop = ast.literal_eval(drop)

ValueError: malformed node or string: [[2420.9805555555554, 2438.4241666666667], [3178.7994444444444, 3454.6875], [3764.9605555555554, 3771.1033333333335]]

In [18]:
x1, x2 = recovery[0]

x1 = float(min(df[(df['time'] > x1 - 1) & (df['time'] < x1 + 1)]['time']))
y1 = float(df[df['time'] == x1]['pressure'])

x2 = float(min(df[(df['time'] > x2 - 1) & (df['time'] < x2 + 1)]['time']))
y2 = float(df[df['time'] == x2]['pressure'])

C:\Users\shara\AppData\Local\Temp\ipykernel_12600\123895989.py:4: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

C:\Users\shara\AppData\Local\Temp\ipykernel_12600\123895989.py:7: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



In [19]:
(x1, y1), (x2, y2)

((2419.980556, 79.022376), (2437.424444, 126.252906))

In [220]:
def log_func(x, A, B, C):
    return A + B * np.log(C * (x - x1) + 1)

In [221]:
import plotly.graph_objects as go

def plot_data(df, selected_file, x1, y1, x2, y2, C):
    B = (y2 - y1)/ np.log(C * (x2 - x1) + 1)
    A = y1

    x_values = np.linspace(x1, x2, 100)
    x_values = x_values[x_values > 0]  
    y_values = log_func(x_values, A, B, C)
    # y_values = np.clip(y_values, df["pressure"].min(), df["pressure"].max())

    fig = px.line(df, x="time", y="pressure", title=f"Visualization of {selected_file} (Reduced Data)", markers=True)
    fig.update_layout(xaxis_title="Time", yaxis_title="Pressure", template="plotly_white") 

    fig.update_layout(
        xaxis=dict(rangeslider=dict(visible=True), type="linear"),
        yaxis=dict(fixedrange=False),
        legend=dict(title="Legend", x=0.99, y=0.99, xanchor="right", yanchor="top")
    )

    fig.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Log Approximation', line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=[x1, x2], y=[y1, y2], mode='markers', name='Selected Points', marker=dict(color='blue', size=10)))

    fig.show()

In [222]:
plot_data(df, 'rofl', x1, y1, x2, y2, 1.9)

In [ ]:
plot_data(df, 'rofl', 3178.7994444444444, 116.4971, 3453.688, 168.9249, 0.0001)вы
# надо подобрать параметры